# Tombstoning Tool Calls

This notebook demonstrates the concept of **tombstoning tool calls** to reduce token consumption in LLM applications.

**Concept**: When tool calls return very long outputs, we can replace them with summarized versions after their first use, reducing token consumption by up to ~95%

**Source**: Mentioned in a recent [Anthropic podcast on YouTube.](https://www.youtube.com/watch?v=XuvKFsktX0Q)

**Two Approaches Demonstrated**:
1. **Naive Tombstoning**: Manual replacement with a short summary
2. **LLM-Derived Tombstoning**: Using an LLM to automatically summarize tool outputs into 5 words or less

## Setup


In [1]:
%pip install openai tiktoken



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-5")

def count_tokens(text):
    return len(encoding.encode(text))

## Initial Tool Call Example


In [3]:
from openai import OpenAI
from getpass import getpass
import json

# Prompt for API key securely
api_key = getpass("Enter your OpenAI API key: ")

# Initialize the OpenAI client
client = OpenAI(api_key=api_key)

In [4]:
# Define a tool that returns a very long response (simulating verbose tool outputs)
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    return f"""{sign}: The celestial bodies are aligning in your favor this week, bringing with them a tide of transformative energy that will wash over every aspect of your life. As the planets dance through the cosmos, they weave intricate patterns that speak directly to your soul's journey. The universe has been conspiring to bring you to this moment, where opportunity and preparation meet in a beautiful cosmic waltz.

In matters of the heart, Venus whispers secrets of connection and understanding. You may find yourself drawn to unexpected places and people, as the stars guide you toward meaningful encounters. Trust your intuition as it navigates these uncharted waters - it knows the way even when your logical mind cannot see the path ahead.

Your professional life is about to experience a significant shift. Jupiter's influence suggests expansion and growth, but remember that all great things require patience and dedication. The seeds you plant now will bear fruit in ways you cannot yet imagine. Stay focused on your long-term vision, even when the daily grind feels overwhelming.

Next Tuesday holds particular significance - you will befriend a baby otter, and this seemingly random encounter will serve as a powerful metaphor for the playful, curious energy you need to embrace. Let this adorable creature remind you that life is meant to be enjoyed, not just endured. The otter's natural wisdom about going with the flow while remaining alert and engaged is exactly the balance you need to cultivate.

Financial prospects look promising, but avoid impulsive decisions. The stars suggest a careful, measured approach to any investments or major purchases. What seems like a golden opportunity on the surface may require deeper examination.

Your health and well-being call for attention. The cosmic energies encourage you to establish better boundaries and prioritize self-care. Remember, you cannot pour from an empty cup. Take time to recharge your physical, emotional, and spiritual batteries.

As this cycle progresses, you'll discover hidden talents and passions you never knew existed. The universe is inviting you to step outside your comfort zone and explore new horizons. Embrace the unknown with courage and curiosity, knowing that the stars are guiding your every step."""


In [5]:
# Initialize conversation with a user query
messages = [
    {"role": "user", "content": "What is my horoscope? I am a Virgo."}
]

In [6]:
# Make API call with tool definitions
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=messages
)

messages += response

# Execute tool calls and append results to messages
for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            horoscope = get_horoscope(json.loads(item.arguments))
            
            messages.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

## Analyze Original Tool Call

Save the original tool call and measure its token count to establish a baseline.


In [7]:
messages

[{'role': 'user', 'content': 'What is my horoscope? I am a Virgo.'},
 ('id', 'resp_04e482205c5545220068ed7cb20c84819681148aa19c3bb3a4'),
 ('created_at', 1760394418.0),
 ('error', None),
 ('incomplete_details', None),
 ('instructions', None),
 ('metadata', {}),
 ('model', 'gpt-5-2025-08-07'),
 ('object', 'response'),
 ('output',
  [ResponseReasoningItem(id='rs_04e482205c5545220068ed7cb2c92c81968dda0fa237e7c512', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
   ResponseFunctionToolCall(arguments='{"sign":"Virgo"}', call_id='call_pT6xXvd3ZKkureVsraFhvpql', name='get_horoscope', type='function_call', id='fc_04e482205c5545220068ed7cb44288819688f63b2b482d3cf9', status='completed')]),
 ('parallel_tool_calls', True),
 ('temperature', 1.0),
 ('tool_choice', 'auto'),
 ('tools',
  [FunctionTool(name='get_horoscope', parameters={'type': 'object', 'properties': {'sign': {'type': 'string', 'description': 'An astrological sign like Taurus or Aquarius'}}, 'required'

In [8]:
import copy

original_tool_call = copy.deepcopy(messages[-1])

In [9]:
original_tool_call

{'type': 'function_call_output',
 'call_id': 'call_pT6xXvd3ZKkureVsraFhvpql',
 'output': '{"horoscope": "{\'sign\': \'Virgo\'}: The celestial bodies are aligning in your favor this week, bringing with them a tide of transformative energy that will wash over every aspect of your life. As the planets dance through the cosmos, they weave intricate patterns that speak directly to your soul\'s journey. The universe has been conspiring to bring you to this moment, where opportunity and preparation meet in a beautiful cosmic waltz.\\n\\nIn matters of the heart, Venus whispers secrets of connection and understanding. You may find yourself drawn to unexpected places and people, as the stars guide you toward meaningful encounters. Trust your intuition as it navigates these uncharted waters - it knows the way even when your logical mind cannot see the path ahead.\\n\\nYour professional life is about to experience a significant shift. Jupiter\'s influence suggests expansion and growth, but remembe

In [10]:
original_tool_call_tokens = count_tokens(original_tool_call["output"])

In [11]:
original_tool_call_tokens

441

## Approach 1: Naive Tombstoning

Manually replace the long tool output with a short, hand-written summary.

In [12]:
new_tool_call    = {
    "type": "function_call_output",
    "call_id": item.call_id,
    "output": json.dumps({
      "horoscope": {
        "sign": "Virgo",
        "horoscope": "You will have good fortune"
      }
    })
}

In [13]:
new_tool_call_tokens = count_tokens(new_tool_call["output"])

In [14]:
new_tool_call_tokens

22

## Approach 2: LLM-Derived Tombstoning

Use a lightweight LLM to automatically summarize the tool output into 5 words or less.

In [15]:

# Use a lightweight model to summarize the output
new_output = client.responses.create(
    model="gpt-5-nano",
    instructions="Rewrite this into 5 words or less",
    input=original_tool_call["output"]
)

In [16]:
llm_summary_tool_call = new_output.output_text

In [17]:
llm_summary_tool_call

'Virgo: transformative week; trust intuition.'

In [18]:
new_llm_summary_tool_call = {
    "type": "function_call_output",
    "call_id": item.call_id,
    "output": json.dumps({
      "horoscope": {
        "sign": "Virgo",
        "horoscope": llm_summary_tool_call
    }
    })
}

In [19]:
new_llm_summary_tool_call

{'type': 'function_call_output',
 'call_id': 'call_pT6xXvd3ZKkureVsraFhvpql',
 'output': '{"horoscope": {"sign": "Virgo", "horoscope": "Virgo: transformative week; trust intuition."}}'}

In [20]:
new_llm_summary_tool_call_tokens = count_tokens(new_llm_summary_tool_call["output"])

In [21]:
new_llm_summary_tool_call_tokens

26

## Results Comparison

Compare token counts between the original and tombstoned tool calls.


In [22]:
# Display original vs tombstoned token counts
(original_tool_call_tokens, new_llm_summary_tool_call_tokens)

(441, 26)

In [23]:
# Calculate token savings
diff_tokens = original_tool_call_tokens - new_llm_summary_tool_call_tokens

In [24]:
diff_tokens

415

### Scale to Multiple Tool Calls

Calculate savings if you have 5 tool calls in your agent.

In [25]:
diff_tokens * 5

2075

In [26]:
# Calculate percentage reduction
percentage_diff = (diff_tokens / original_tool_call_tokens) * 100

In [27]:
percentage_diff

94.10430839002267